In [3]:
import plotly.express as px
import pandas as pd
import numpy as np
from faker import Faker
from dash import dcc
import dash
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.utilities import JsCode
from folium.features import GeoJsonPopup
from folium.plugins import TimeSliderChoropleth
import branca
import datetime


In [7]:
# Set random seed for reproducibility
np.random.seed(0)

# Initialize Faker to generate fake data
fake = Faker()

# Define the number of rows in the dataset
num_rows = 15000

# Create a DataFrame to store the fake dataset
practice_df = pd.DataFrame()

# Create opening hour variable
opening_hour_range = range(7, 9)
opening_hour = np.random.choice(opening_hour_range, size=num_rows)

# Create closing hour variable
closing_hour_range = range(17, 22)
closing_hour = np.random.choice(closing_hour_range, size=num_rows)

# Create DaysofWeek variable
daysofweek_categories = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daysofweek = [np.random.choice(daysofweek_categories, size=np.random.randint(1, len(daysofweek_categories) + 1), replace=False) for _ in range(num_rows)]

# Add DaysofWeek to DataFrame
practice_df['daysofweek'] = daysofweek

# Add the opening hour variable to the DataFrame
practice_df['opening_hour'] = opening_hour

# Add the closing hour variable to the DataFrame
practice_df['closing_hour'] = closing_hour

# Add opacity variable to data frame
practice_df['opacity'] = opacity

# Generate aed location variable
min_lat, max_lat = 49.9, 51.1  # Latitude boundaries of Belgium
min_lon, max_lon = 3, 6    # Longitude boundaries of Belgium
practice_df['Latitude'] = np.random.uniform(min_lat, max_lat, size=num_rows)
practice_df['Longitude'] = np.random.uniform(min_lon, max_lon, size=num_rows)

aed_placement_categories = ['outside', 'inside', 'difficult']
aed_placement = np.random.choice(aed_placement_categories, size=num_rows)
practice_df['aed_placement'] = aed_placement

# Define buffer distances
buffer_distances = {
    'outside': 200,  # ~200 meters in degrees
    'inside': 150,   # ~150 meters in degrees
    'difficult': 125  # ~125 meters in degrees
}

practice_df['buffer_distance'] = practice_df['aed_placement'].map(buffer_distances)

# Add unique identifier for each row
practice_df['id'] = practice_df.index

# Initialize a dictionary to hold all the new columns
new_columns_dict = {}

# Pre-calculate colors based on all possible combinations of days and hours
for day in daysofweek_categories:
    for hour in range(24):
        column_name = f'color_{day}_{hour}'
        condition = (
            (practice_df['daysofweek'].apply(lambda x: day in x)) & 
            (practice_df['opening_hour'] <= hour) & 
            (practice_df['closing_hour'] >= hour)
        )
        new_columns_dict[column_name] = np.where(condition, 'green', 'red')

# Create a DataFrame from the dictionary
color_df = pd.DataFrame(new_columns_dict)

# Concatenate the new columns to the original DataFrame in a single operation
practice_df = pd.concat([practice_df, color_df], axis=1)

# Convert DataFrame to GeoDataFrame
practice_gdf = gpd.GeoDataFrame(
    practice_df,
    geometry=[Point(xy) for xy in zip(practice_df['Longitude'], practice_df['Latitude'])],
    crs="EPSG:4326"  # WGS84 coordinate system
)

def generate_folium_map(selected_day):
    m = folium.Map(location=[50.5, 4.3517], zoom_start=8)

    styledict = {}

    for hour in range(24):
        styledict[hour] = {}
        for idx, row in practice_df.iterrows():
            color_column = f'color_{selected_day}_{hour}'
            style_dict = {
                'color': str(row[color_column]),
                'fillColor': str(row[color_column]),
                'fillOpacity': float(row['opacity'])  # Ensure opacity is a float
            }
            styledict[hour][int(row['id'])] = style_dict
    print(f'StyleDict for {selected_day}:', styledict)


print(practice_df.head())



                              daysofweek  opening_hour  closing_hour  \
0                               [Sunday]             7            17   
1  [Thursday, Wednesday, Monday, Friday]             8            20   
2                             [Saturday]             8            18   
3              [Tuesday, Monday, Sunday]             7            17   
4                               [Sunday]             8            19   

    opacity   Latitude  Longitude aed_placement  buffer_distance  id  \
0  0.763139  50.894709   3.078928        inside              150   0   
1  0.615854  50.673781   4.976167        inside              150   1   
2  0.121289  49.931934   4.764849       outside              200   2   
3  0.797758  50.946637   3.796990     difficult              125   3   
4  0.114180  49.921270   4.772946        inside              150   4   

  color_Monday_0  ... color_Sunday_14 color_Sunday_15 color_Sunday_16  \
0            red  ...           green           green        

In [6]:
TimeSliderChoropleth(
        data=practice_gdf.set_index('id').to_json(),
        styledict=styledict
    ).add_to(m)
return m.get_root().render()

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.RadioItems(
        id='day-radioitems',
        options=[{'label': day, 'value': day} for day in daysofweek_categories],
        value='Monday',
        inline=True
    ),
    html.Iframe(
        id='folium-map',
        srcDoc=generate_folium_map('Monday'),
        width='100%',
        height='600px'
    )
])

@app.callback(
    Output('folium-map', 'srcDoc'),
    [Input('day-radioitems', 'value')]
)
def update_map(selected_day):
    return generate_folium_map(selected_day)

if __name__ == '__main__':
    app.run_server(debug=True)

TypeError: Object of type ndarray is not JSON serializable